In [1]:
import os
import numpy as np
from datasets import Dataset
from extract_sep_data import load_sep_dataset
import json
from struq import generate_training_data, SupervisedDataset, load_jsonl
from transformers import AutoModelForCausalLM, AutoTokenizer, Qwen3MoeConfig, Qwen3MoeForCausalLM

/Users/vblack/opt/miniconda3/envs/qwen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
data_path = 'datasets/alpaca_data_with_input_100.jsonl'
model_path = "outputs/struq_training/qwen3-0.6b"
base_model_name = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForCausalLM.from_pretrained(base_model_name)

In [5]:
len(tokenizer)

151669

In [14]:
import torch

def inference(model, tokenizer, input):
    input_text = input['instruction'] + "\n" + input['data']
    messages = [
            {"role": "user", "content": input_text}
        ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True, 
        enable_thinking=True
    )
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=32768)
    # decode
    output_ids = outputs[0][len(inputs.input_ids[0]):].tolist()
    try:
        index = len(output_ids) - output_ids[::-1].index(151668)
    except ValueError:
        index = 0
    thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True)
    content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")
    return content, outputs

example = probe_data[0]
response, outputs = inference(model, tokenizer, example)
print(response)

The **Sarbanes-Oxley Act (SOX)**, enacted in 2001, was a landmark piece of legislation aimed at protecting investors from corporate fraud by enhancing accountability in financial reporting. It mandated stricter reforms to corporate financial disclosures, requiring companies to adopt more transparent and audited accounting practices. Key legal implications include:  
1. **Increased Oversight**: SOX required companies to implement internal controls and regular audits of financial reports, reducing the risk of fraudulent accounting activities.  
2. **Accountability Measures**: It mandated that executives and management be held accountable for financial misstatements, ensuring that companies are held responsible for their actions.  
3. **Protecting Shareholders**: By improving transparency, SOX sought to prevent investors from being misled by companies that failed to disclose errors or misrepresent their financial status.  

Regarding the **composition of a book**, a **book** typically ref

In [13]:
example_2 = probe_data[1]
response_2, outputs_2 = inference(model, tokenizer, example_2)
response_2

"The **Sarbanes-Oxley Act (SOX)**, enacted in 2002, was designed to protect investors from financial fraud by corporations. It requires **strong corporate financial reporting standards** and mandates **rigorous internal controls** to prevent fraudulent accounting practices. This act aimed to improve transparency and accountability in corporate financial reporting, which in turn helped investors make more informed decisions.\n\nA **book** in this context refers to a **financial document or record** used to present a company's financial activities. It typically includes data such as revenue, expenses, balances, and other financial statements. The primary purpose of a book is to **document and present financial information** to stakeholders, investors, and regulators."

In [2]:
from datasets import load_dataset

ds = load_dataset("tatsu-lab/alpaca")

/Users/vblack/opt/miniconda3/envs/qwen/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
ds['train'].to_json("datasets/alpaca_data.json")

Creating json from Arrow format: 100%|██████████| 53/53 [00:00<00:00, 124.30ba/s]


48735970